In [10]:
import cv2
import numpy as np

#Load YOLO
net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

#Load input image
image = cv2.imread('img/lake.jpg')
height, width, _ = image.shape

#Preprocess input image
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

#Forward pass through the network
output_layers_names = net.getUnconnectedOutLayersNames()
layer_outputs = net.forward(output_layers_names)

#Process detection results
boxes = []
confidences = []
class_ids = []

for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

#Non max supression to remove redundant overlapping boxes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(classes), 3))

#Draw bounding boxes and labels
for i in indexes.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i], 2))
    color = colors[i]
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
    cv2.putText(image, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

#Display the output image
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

# Load YOLO (same as before)
net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
  classes = f.read().splitlines()

# Open video capture
cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')

# Loop through video frames
while cap.isOpened():
  # Capture frame-by-frame
  ret, frame = cap.read()
  
  # Check if frame is read correctly
  if not ret:
      print("Can't receive frame (stream end?). Exiting...")
      break

  height, width, _ = frame.shape

  # Preprocess frame (same as before)
  blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)

  # Forward pass through the network (same as before)
  output_layers_names = net.getUnconnectedOutLayersNames()
  layer_outputs = net.forward(output_layers_names)

  # Process detection results (same as before)
  boxes = []
  confidences = []
  class_ids = []
  for output in layer_outputs:
    for detection in output:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]
      if confidence>0.5:
          center_x = int(detection[0] * width)
          center_y = int(detection[1] * height)
          w = int(detection[2] * width)
          h = int(detection[3] * height)
          x = int(center_x - w / 2)
          y = int(center_y - h / 2)
          
          boxes.append([x, y, w, h])
          confidences.append(float(confidence))
          class_ids.append(class_id)

  # Non max supression (same as before)
  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

  # Draw bounding boxes and labels (same as before)
  font = cv2.FONT_HERSHEY_PLAIN
  colors = np.random.uniform(0, 255, size=(len(classes), 3))
  for i in indexes.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i], 2))
    color = colors[i]
    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

  # Display the resulting frame and clear previous window (new addition)
  cv2.imshow('SST Foyer Video', frame)
    
 # Exit if 'q' key is pressed
  if cv2.waitKey(1) == ord('q'):
      break


# Release capture and close windows
#cap.release()
##cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

def play_video():
    video_path = filedialog.askopenfilename()
    if not video_path:
        return

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Video', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def detect_objects():
    video_path = filedialog.askopenfilename()
    if not video_path:
        return

    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        cv2.imshow('Object Detection', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def welcome_message():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}! What operation would you like to perform?")

def end_operation():
    root.destroy()

root = tk.Tk()
root.title("Video Operations")

label_name = tk.Label(root, text="Enter your name:")
label_name.pack()

entry_name = tk.Entry(root)
entry_name.pack()

button_welcome = tk.Button(root, text="Welcome", command=welcome_message)
button_welcome.pack()

button_watch = tk.Button(root, text="Watch Video", command=play_video)
button_watch.pack()

button_detect = tk.Button(root, text="Detect Objects", command=detect_objects)
button_detect.pack()

button_end = tk.Button(root, text="End Operation", command=end_operation)
button_end.pack()

root.mainloop()
